In [ ]:
combined_keyframes_metadata_filename = None
keyframes_metadata_dir = None


In [ ]:
import os
import json

dir_path = os.getcwd()

In [ ]:
data_extraction_path = f'{dir_path}/data_extraction'
dataset_path = f'{data_extraction_path}/dataset/AIC_Video'

if not combined_keyframes_metadata_filename:
    combined_keyframes_metadata_filename = 'keyframes_metadata.json'
    
if not keyframes_metadata_dir:
    keyframes_metadata_dir = f'{dir_path}/data_extraction/transnet/Keyframes_Metadata'

# Combine keyframe metadata

In [ ]:

def combine_json_files(directory, output_file):
    combined_data = {}

    # Load existing data if the file exists
    if os.path.exists(output_file):
        with open(output_file, 'r') as existing_file:
            combined_data = json.load(existing_file)

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)

            # Read the contents of the JSON file
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Update combined_data with new data (overriding if keys exist)
            combined_data.update(data)

    # Write combined_data to the output JSON file
    with open(output_file, 'w') as outfile:
        json.dump(combined_data, outfile)
    
    print(f'Combined keyframe metadata successful: {output_file}')
    

combine_json_files(keyframes_metadata_dir,
                   combined_keyframes_metadata_filename)

Combined keyframe metadata successful: keyframes_metadata.json
